In [ ]:
!pip install music21
!pip install glob
!pip install tqdm
!pip install tensorflow
!pip install sklearn
!pip install random

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [ ]:
!pip install numpy

In [ ]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Bidirectional

def read_files(file):
    notes = []
    try:
        # parse the midi file
        midi = converter.parse(file)

        # separate all instruments from the file
        instrmt = instrument.partitionByInstrument(midi)

        for part in instrmt.parts:
            # fetch data only of Piano instrument
            if 'Piano' in str(part):
                notes_to_parse = part.recurse()

                # iterate over all the parts of sub stream elements
                # check if element's type is Note or chord
                # if it is chord split them into notes
                for element in notes_to_parse:
                    if isinstance(element, note.Note):
                        notes.append(str(element.pitch))
                    elif isinstance(element, chord.Chord):
                        notes.append('.'.join(str(n) for n in element.normalOrder))
    except Exception as e:
        print(f"Error processing {file}: {e}")

    # return the list of notes
    return notes


#retrieve paths recursively from inside the directories/files
file_path=["/content/drive/MyDrive/music_genaration/Rock"]
all_files=glob.glob(file_path[0]+'/*.mid',recursive=True)

#reading each midi file
#notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])
notes_array = np.array([read_files(i) for i in tqdm(all_files, position=0, leave=True)])

#unique notes
notess = sum(notes_array,[])
print(notess)
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))
print(freq_notes)

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))
print(ind2note)

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))
print(note2ind)

#timestep
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

import numpy as np
from sklearn.model_selection import train_test_split

x_new = np.array(x)
y_new = np.array(y)

# Reshape input and output for the model
x_new = np.reshape(x_new, (len(x_new), timesteps, 1))
y_new = np.reshape(y_new, (-1, 1))


# Split the input and value into training and testing sets
# 80% for training and 20% for testing sets
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.8, random_state=42)

#create the model
"""model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()"""


model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(x_new.shape[1], x_new.shape[2])))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512)))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(note2ind), activation='softmax'))
model.summary()






#compile the model using Adam optimizer
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model on training sets and validate on testing sets

model.fit(
    x_train, y_train,
    batch_size=128, epochs=100,
    validation_data=(x_test, y_test)
)

#save the model for predictions
model.save("/content/drive/MyDrive/music_genaration/trained_Rock_model.h5")

#load the model
model=load_model("/content/drive/MyDrive/music_genaration/trained_Rock_model.h5")
#generate random index
#index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
#music_pattern = x_test[index]


index = np.random.randint(0, len(x_test) - 1)
music_pattern = x_test[index]

out_pred=[] #it will store predicted notes

for i in range(200):
    music_pattern_reshaped = music_pattern.reshape(1, len(music_pattern), 1)
    pred_index = np.argmax(model.predict(music_pattern_reshaped))
    out_pred.append(ind2note[pred_index])
    music_pattern = np.append(music_pattern, pred_index)
    music_pattern = music_pattern[1:]


#iterate till 200 note is generated
"""for i in range(200):

  #reshape the music pattern
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)

  #get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))
  #get the note using predicted index and
  #append to the output prediction list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)

  #update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]"""

output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)

    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)

  else:
    #cast the pattern to Note object apply the offset and
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='new _music.mid')


  0%|          | 0/509 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'\xff Sequenced 1999 by'>; getting generic Instrument
  warnings.warn(
  2%|▏         | 12/509 [01:05<43:55,  5.30s/it]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Distorsi\xf3n'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Soundtrack/El\xe9ctrica'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine in

Error processing /content/drive/MyDrive/music_genaration/Rock/John_Paul_Young_-_Love_is_in_the_Air.mid: badly formed midi string: missing leading MTrk


 95%|█████████▍| 483/509 [41:52<01:59,  4.58s/it]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=17, channel=None, data=b'Copyright \xa9  2000 VJ Miele'>; getting generic Instrument
  warnings.warn(
100%|██████████| 509/509 [44:12<00:00,  5.21s/it]
<ipython-input-3-6431067df151>:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  notes_array = np.array([read_files(i) for i in tqdm(all_files, position=0, leave=True)])


['0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '5.9', 'F2', '4.7', '2.5', '0.4', '5.9', '4.7', '2.5', 'F2', 'E3', 'C3', 'C3', 'C3', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '5.9', 'F2', '4.7', '2.5', '0.4', '5.9', '4.7', '2.5', 'E3', 'C3', 'E2', 'A2', '9.0', 'F3', '10.2', '10.2', '10.2.5', '0.4', 'G3', '10.0.4', 'F3', 'A3', 'C3', 'F3', '5.9', '9.2', 'F3', 'F3', 'D3', 'A3', 'D3', 'G3', 'B-3', 'D3', 'F3', '2.7', '10.0', '4.7', '4.7', 'C3', 'A3', 'E3', 'A3', '9.2', 'F3', 'F3', '2.5.9', '9.10.2.5', '9.10.2.5', '5.9.0', 'F4', 'E4', 'F4', '0.2.7', '7.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '0.4.7', '0.2.4', '7.9.2', '9.11.2', '5.9', 'F2', '4.7', '2.5', '0.4', '5.9', '4.7', 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 20ms/step


'new _music.mid'